In [1]:
import numpy as np
import warnings
import torch

### Split Version. OLD. See the newer version "Viterbi_v2" for a more general formulation that allows you to play around with various truth configurations of the constraints.

# Model Overview.

We're going with a 3-state HMM with 2-state emissions:

1. Uniform transition matrix
2. Uniform initial distribution
3. Binary emissions with emission matrix: $\begin{bmatrix} .8 & .2\\ .2 & .8\\ .5 & .5 \end{bmatrix}$

We're going to additionally impose the constraint that state $0$ must happen before state $1$. To make things compact, we're going use just a single mediator $M$ that tracks if state $0$ has happened yet. Then, if $X_t = 1$ but $M_t = 0$ (so $1$ happens before $0$) the constraint is violated.

## Unconstrained vs Constrained

By design, in the unconstrained case the optimal hidden state sequence given an observation sequence will just be a copy, That is, all hidden sequence have equal prior probability, and since state $0$ is most likely to emit $0$ and state $1$ is most likely to emit $1$, the MAP given observation sequence $O$ is just $S^* = O$.

**Importantly, state $2$ will never be chosen in the unconstrained case**. However, this changes if we introduce the constraint that state $0$ must happen before $1$. Given an observation sequence like $[1,0,0,1,\cdots]$, we will actually pick state $2$, which is more likely than state $0$ to emit $1$, until we encounter state $0$. Afterwards, the optimal hidden sequence will just be a copy of the remaining observation sequence. 

EDIT: If the initial sequence of $1$'s is longer than $3$ ($.5^3 \leq .2 $), the model could be incentivized to assign a $0$-state so that it can maximize the probability remaining sequence of $1$'s by using $P(1|1) = .8$ rather than $P(1|2) = .5$. The model will assign $2$ as long as the initial sequence of $1$'s is at most 2.


## The Augmented Model
Let $M_t$ be the binary tracker for if state $0$ has occured in the chain yet: $M_t = M_{t-1} 1_{X_t = 0}$. Let $C_t = 1 - (1_{M_t = 0} 1_{X_t = 1})$: ie. the indicator for the negation of "we haven't encountered state 0 yet but we're in state 1". Enforcing state 0 to come before state 1 is equivalent to conditioning on $C_t = 1$ for all $t$. Since I'm lazy, I'm going to blow up the state space $(M,X)$ and emissions $(Y,C)$ so I can use existing code for Viterbi.



### Augmented Model Ordering Convention
Our augmented state space has 6 states corresponding to $(X,M)$, and our modified emissions matrix will be a $6 \times 4$ matrix: 6 hiddens states and $(Y,C)$ emissions - 4 in total. We'll order the state and emission tuples by the dictionary order. 

1. Transition matrix. Row/cols are indexed in dictionary order: $(0,0), (1,0),\cdots, (2,1)$. The first index is the orginal hidden state, the second is the tracker for if state $0$ has happened yet. Note the state $(0,0)$ is impossible and we set it as an absorbing state.
2. Emission matrix. The rows of the emission matrix are indexed as $(0,0),(1,0),\cdots,(1,1)$ where the first index is the origina emission and the second the truth value of the constraints.
3. Observation. Since we condition on the constraints being true, we will have either $(0,1)$ or $(1,1)$. This corresponds to an index of 3 or 4 in the dictionary order.

### Code Acknowledgement
The code for the Viterbi algorith was copied from this [blog post](https://www.audiolabs-erlangen.de/resources/MIR/FMP/C5/C5S3_Viterbi.html)


In [2]:
def viterbi(A, C, B, O):
    """Viterbi algorithm for solving the uncovering problem

    Notebook: C5/C5S3_Viterbi.ipynb

    Args:
        A (np.ndarray): State transition probability matrix of dimension I x I
        C (np.ndarray): Initial state distribution  of dimension I
        B (np.ndarray): Output probability matrix of dimension I x K
        O (np.ndarray): Observation sequence of length N

    Returns:
        S_opt (np.ndarray): Optimal state sequence of length N
        D (np.ndarray): Accumulated probability matrix
        E (np.ndarray): Backtracking matrix
    """
    I = A.shape[0]  # Number of states
    N = len(O)  # Length of observation sequence

    # Initialize D and E matrices
    D = np.zeros((I, N))
    E = np.zeros((I, N - 1)).astype(np.int32)
    D[:, 0] = np.multiply(C, B[:, O[0]])

    # Compute D and E in a nested loop
    for n in range(1, N):
        for i in range(I):
            temp_product = np.multiply(A[:, i], D[:, n - 1])
            D[i, n] = np.max(temp_product) * B[i, O[n]]
            E[i, n - 1] = np.argmax(temp_product)

    # Backtracking
    S_opt = np.zeros(N).astype(np.int32)
    S_opt[-1] = np.argmax(D[:, -1])
    for n in range(N - 2, -1, -1):
        S_opt[n] = E[int(S_opt[n + 1]), n]

    return S_opt, D, E

Note that a transition to $(0,0)$ is impossible, and if we transition to original state $0$ then we will always transition to augmented state $(0,1)$

In [3]:
tmat = np.array(
    [
        [
            1,
            0,
            0,
            0,
            0,
            0,
        ],  # impossible state. just set it to an absorbing one that should never occur.
        [0, 1 / 3, 1 / 3, 1 / 3, 0, 0],
        [0, 1 / 3, 1 / 3, 1 / 3, 0, 0],
        [0, 0, 0, 1 / 3, 1 / 3, 1 / 3],
        [0, 0, 0, 1 / 3, 1 / 3, 1 / 3],
        [0, 0, 0, 1 / 3, 1 / 3, 1 / 3],
    ]
)
# row/cols are indexed in dictionary order: (0,0), (1,0),...(2,1)
#

emit = np.array(
    [
        [0, 0, 0.8, 0.2],
        [0.2, 0.8, 0, 0],
        [0, 0, 0.5, 0.5],
        [0, 0, 0.8, 0.2],
        [0, 0, 0.2, 0.8],
        [0, 0, 0.5, 0.5],
    ]
)
tmat

array([[1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.33333333, 0.33333333, 0.33333333, 0.        ,
        0.        ],
       [0.        , 0.33333333, 0.33333333, 0.33333333, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.33333333, 0.33333333,
        0.33333333],
       [0.        , 0.        , 0.        , 0.33333333, 0.33333333,
        0.33333333],
       [0.        , 0.        , 0.        , 0.33333333, 0.33333333,
        0.33333333]])

Intialize with uniform distribution over all original hidden states. Note that if we initialize to $0$, then we also initialize to $Y = 1$, which is why the initial distribution is shifted below. 

In [4]:
init_prob = np.array([0, 1 / 3, 1 / 3, 1 / 3, 0, 0])
init_prob

array([0.        , 0.33333333, 0.33333333, 0.33333333, 0.        ,
       0.        ])

We observe the original sequence below. Since we also condition on the constraints being true, we shfit by two ie. $i \rightarrow (i,1)$

In [5]:
og_obs = [1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1]
shift_obs = np.array(og_obs) + 2
shift_obs

array([3, 3, 3, 3, 2, 3, 2, 3, 3, 2, 3])

In [6]:
# Define model parameters
A = tmat
C = init_prob
B = emit
O = shift_obs

# O = np.array([1]).astype(np.int32)
# O = np.array([1, 2, 0, 2, 2, 1]).astype(np.int32)

# Apply Viterbi algorithm
S_opt, D, E = viterbi(A, C, B, O)

# Now convert expanded states into original states

S_convert = S_opt % 3

# Number of initial 1 emissions is greater than 2.
print("Observation sequence:   O = ", O)
print("Optimal Augmented Hidden State: S_aug = ", S_opt)
print("Optimal Original Hidden State: S = ", S_convert)

Observation sequence:   O =  [3 3 3 3 2 3 2 3 3 2 3]
Optimal Augmented Hidden State: S_aug =  [3 4 4 4 3 4 3 4 4 3 4]
Optimal Original Hidden State: S =  [0 1 1 1 0 1 0 1 1 0 1]


In [7]:
og_obs = [1, 1, 0, 1, 0, 1, 1, 0, 1]
shift_obs = np.array(og_obs) + 2
shift_obs

array([3, 3, 2, 3, 2, 3, 3, 2, 3])

In [8]:
# Define model parameters
A = tmat
C = init_prob
B = emit
O = shift_obs

# O = np.array([1]).astype(np.int32)
# O = np.array([1, 2, 0, 2, 2, 1]).astype(np.int32)

# Apply Viterbi algorithm
S_opt, D, E = viterbi(A, C, B, O)

# Now convert expanded states into original states

S_convert = S_opt % 3

# Number of initial 1 emissions is greater than 2.
print("Observation sequence:   O = ", O)
print("Optimal Augmented Hidden State: S_aug = ", S_opt)
print("Optimal Original Hidden State: S = ", S_convert)

Observation sequence:   O =  [3 3 2 3 2 3 3 2 3]
Optimal Augmented Hidden State: S_aug =  [2 2 3 4 3 4 4 3 4]
Optimal Original Hidden State: S =  [2 2 0 1 0 1 1 0 1]


Here is a weakness in the current formulation. Let say we now want to do inference with the knowledge that the constraint is false: state 1 happens before state 0. From our setup, this is equivalent to conditioning on the event that at least one $C_t =0$. But this is not so easy since it doesn't translate into a single assignment to the augmented emissions $C_t$. 